In [14]:
import eeweather as ew
import datetime as dt
import pandas as pd
import pytz

In [38]:
def get_hourly_weather(zip, local_tz, site, start_date, stop_date, filepath):
    """
    function to get hourly weather data from eeweather
    requires: zipcode, local timezone, site ID or name, start/stop dates, filepath
    returns: ranked weather stations, warnings, baseline and post weather data
    """
    lat, lon = ew.zcta_to_lat_long(zip)

    ranked_stations = ew.rank_stations(lat, lon,
                                   is_tmy3=True,
                                   match_ba_climate_zone = True,
                                   minimum_quality = "medium")
    
    station, warnings = ew.select_station(ranked_stations)
    
    data = station.load_isd_hourly_temp_data(start_date, stop_date)[0].reset_index()
    data = data.rename(columns = {data.columns[0]:"timestamp_UTC", data.columns[1]:"outdoor_air_temp_degC"})
    data["outdoor_air_temp_degF"] = ((data["outdoor_air_temp_degC"] * (9/5)) + 32)
    data["timestamp_local"] = data["timestamp_UTC"].dt.tz_convert(local_tz)

    data.to_csv(f"{filepath}site{site}_zip{zip}_isdstation{station}_{start_date.strftime('%m%d%Y')}-{stop_date.strftime('%m%d%Y')}"+".csv",
                index=False)
    ranked_stations.head(10).to_csv(f"{filepath}/metadata/metadata_zip{zip}_isdstation{station}"+".csv",
                index=False)

In [40]:
get_hourly_weather("99574",
                   "America/Anchorage",
                   "CordovaAK",
                   start_date = dt.datetime(2015, 1, 1, tzinfo=pytz.UTC),
                   stop_date = dt.datetime(2023, 6, 14, tzinfo=pytz.UTC),
                   filepath = "/Users/rose775/Library/CloudStorage/OneDrive-PNNL/DHP Field Validation/Cordova_AK/hist_weather_data/")
